# Assigning Regions to Sale Representatives at Pfizer Turkey 


In [46]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
import json
import collections
import copy

assignment = pd.read_csv("assignment.csv")
distances = pd.read_csv("brick_rp_distances.csv")
workload = pd.read_csv("bricks_index_values.csv")

In [47]:
def str_to_list(to_cast):
    return json.loads(to_cast)

In [48]:
def create_init_assign_matrix(assignments):
    assignment_d = {}
    for i in range(4):
        x = assignment[assignment["SR#"] == (i+1)]
        for brick in str_to_list(x["Bricks_Assigned"][i]):
            assignment_d[brick] = i + 1
    sorted_dict = dict(sorted(assignment_d.items()))
    matrix = [[int((i+1) == sorted_dict[k])for i in range(4)] for k in sorted_dict.keys()]
    return matrix

In [49]:
def create_index_value_matrix(bricks_index_values):
    index_dict = bricks_index_values.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(index_dict.items()))
    return [[float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0])] for index in sorted_dict.keys()]

In [50]:
def create_distances_matrix(distances):
    distances_dict = distances.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(distances_dict.items()))
    return [sorted_dict[index] for index in sorted_dict.keys()]

## MIP style

In [51]:
# Create the mip solver with the CP-SAT backend.
solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("Problème lors de la création du solveur")

# Values for constraints
num_sr = 4
min_workload = 0.8
max_workload = 1.2
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
distances_matrix = create_distances_matrix(distances)
hq_bricks = [[0,3],[1,13],[2,15],[3,21]]
num_bricks = len(distances_matrix)

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = solver.BoolVar(f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    solver.Add(solver.Sum([x[brick, sr] for sr in range(num_sr)]) == 1)

# The cental brick for an SR cannot change
solver.Add(solver.Sum([x[hq[1], hq[0]] for hq in hq_bricks]) == 4)

# Balance workload (min and max)
for sr in range(num_sr):
    solver.Add(solver.Sum([index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)]) >= min_workload)
    solver.Add(solver.Sum([index_value_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr)]) <= max_workload)

In [52]:
# Minimizing the distance
objective_terms = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms.append(distances_matrix[brick][sr] * x[brick, sr])
solver.Minimize(solver.Sum(objective_terms))

"""
# Minimizing rassignments
objective_terms_2 = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms_2.append(1/2*(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr]))
solver.Minimize(solver.Sum(objective_terms_2))
"""

'\n# Minimizing rassignments\nobjective_terms_2 = []\nfor brick in range(num_bricks):\n    for sr in range(num_sr):\n        objective_terms_2.append(1/2*(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr]))\nsolver.Minimize(solver.Sum(objective_terms_2))\n'

In [53]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()
print(status)

Solving with SCIP 9.0.0 [LP solver: Glop 9.11]
0


In [54]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if x[brick, sr].solution_value():
                print(
                    f"Brick {brick + 1} assigned to SR {sr + 1}."
                    + f" Cost: {distances_matrix[brick][sr]}"
                )
else:
    print("No solution found.")


Total cost = 154.08

Brick 1 assigned to SR 4. Cost: 21.12
Brick 2 assigned to SR 4. Cost: 17.33
Brick 3 assigned to SR 4. Cost: 12.25
Brick 4 assigned to SR 1. Cost: 0.0
Brick 5 assigned to SR 1. Cost: 3.07
Brick 6 assigned to SR 1. Cost: 1.22
Brick 7 assigned to SR 1. Cost: 2.8
Brick 8 assigned to SR 1. Cost: 2.87
Brick 9 assigned to SR 1. Cost: 3.8
Brick 10 assigned to SR 3. Cost: 4.37
Brick 11 assigned to SR 3. Cost: 2.97
Brick 12 assigned to SR 1. Cost: 21.99
Brick 13 assigned to SR 2. Cost: 3.28
Brick 14 assigned to SR 2. Cost: 0.0
Brick 15 assigned to SR 3. Cost: 1.11
Brick 16 assigned to SR 3. Cost: 0.0
Brick 17 assigned to SR 3. Cost: 1.08
Brick 18 assigned to SR 2. Cost: 0.77
Brick 19 assigned to SR 1. Cost: 11.13
Brick 20 assigned to SR 1. Cost: 17.49
Brick 21 assigned to SR 4. Cost: 25.43
Brick 22 assigned to SR 4. Cost: 0.0


## CPModel style

In [56]:
model = cp_model.CpModel()

In [57]:
# Values for constraints
num_sr = 4
min_workload = 0.8
max_workload = 1.2
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
index_value_matrix_scaled = copy.deepcopy(index_value_matrix)
for i in range(len(index_value_matrix)):
    for j in range(len(index_value_matrix[0])):
        index_value_matrix_scaled[i][j] = int(index_value_matrix[i][j] * 10000)
distances_matrix = create_distances_matrix(distances)
distances_matrix_scaled = copy.deepcopy(distances_matrix)
for i in range(len(distances_matrix)):
    for j in range(len(distances_matrix[0])):
        distances_matrix_scaled[i][j] = int(distances_matrix[i][j] * 100)
hq_bricks = [[0,3],[1,13],[2,15],[3,21]]
num_bricks = len(distances_matrix)

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = model.new_bool_var(f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    model.add_exactly_one(x[brick, sr] for sr in range(num_sr))

# The cental brick for an SR cannot change
hq_vars = []
for hq in hq_bricks:
    hq_vars.append(x[hq[1],hq[0]])
model.add(sum(hq_vars) == 4)

# Balance workload (min and max)
for sr in range(num_sr):
    model.add(sum(index_value_matrix_scaled[brick][sr] * x[brick,sr] for brick in range(num_bricks)) >= int(min_workload * 10000))
    model.add(sum(index_value_matrix_scaled[brick][sr] * x[brick,sr] for brick in range(num_bricks)) <= int(max_workload * 10000))

In [58]:
objective_terms = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms.append(distances_matrix_scaled[brick][sr] * x[brick, sr])
model.minimize(sum(objective_terms))

In [59]:
solver = cp_model.CpSolver()
status = solver.solve(model)
print(status)

4


In [60]:
# Number of reassignments (upper bound for 2nd opti)
new_objective = 0

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Total cost = {solver.objective_value/100}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if solver.boolean_value(x[brick, sr]):
                new_objective += (1 - 2*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])
                print(
                    f"Brick {brick + 1} assigned to SR {sr + 1}."
                    + f" Cost = {distances_matrix[brick][sr]}"
                )
else:
    print("No solution found.")

Total cost = 154.6

Brick 1 assigned to SR 4. Cost = 21.12
Brick 2 assigned to SR 4. Cost = 17.33
Brick 3 assigned to SR 4. Cost = 12.25
Brick 4 assigned to SR 1. Cost = 0.0
Brick 5 assigned to SR 1. Cost = 3.07
Brick 6 assigned to SR 1. Cost = 1.22
Brick 7 assigned to SR 1. Cost = 2.8
Brick 8 assigned to SR 1. Cost = 2.87
Brick 9 assigned to SR 1. Cost = 3.8
Brick 10 assigned to SR 3. Cost = 4.37
Brick 11 assigned to SR 2. Cost = 3.51
Brick 12 assigned to SR 1. Cost = 21.99
Brick 13 assigned to SR 2. Cost = 3.28
Brick 14 assigned to SR 2. Cost = 0.0
Brick 15 assigned to SR 3. Cost = 1.11
Brick 16 assigned to SR 3. Cost = 0.0
Brick 17 assigned to SR 3. Cost = 1.08
Brick 18 assigned to SR 2. Cost = 0.77
Brick 19 assigned to SR 1. Cost = 11.13
Brick 20 assigned to SR 1. Cost = 17.49
Brick 21 assigned to SR 4. Cost = 25.43
Brick 22 assigned to SR 4. Cost = 0.0


In [61]:
# Adding hints as constraints
for brick in range(num_bricks):
    for sr in range(num_sr):
        model.add_hint(x[brick, sr], solver.Value(x[brick, sr]))

In [62]:
# Num of reassignments should be at least lower than the upper bound
for brick in range(num_bricks):
    model.add(sum(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr] for sr in range(num_sr)) <= round(new_objective))

In [63]:
# Minimizing reassignments
objective_terms_2 = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms_2.append(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])
model.minimize(sum(objective_terms_2))

In [64]:
solver = cp_model.CpSolver()
status = solver.solve(model)
print(status)

4


In [66]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Total cost = {solver.objective_value/2}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if solver.boolean_value(x[brick, sr]):
                print(
                    f"Brick {brick + 1} assigned to SR {sr + 1}."
                    + f" Cost = {(1 - 2*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])}"
                )
else:
    print("No solution found.")

Total cost = 1.0

Brick 1 assigned to SR 4. Cost = 0
Brick 2 assigned to SR 4. Cost = 0
Brick 3 assigned to SR 4. Cost = 0
Brick 4 assigned to SR 1. Cost = 0
Brick 5 assigned to SR 1. Cost = 0
Brick 6 assigned to SR 1. Cost = 0
Brick 7 assigned to SR 1. Cost = 0
Brick 8 assigned to SR 1. Cost = 0
Brick 9 assigned to SR 3. Cost = 0
Brick 10 assigned to SR 3. Cost = 1
Brick 11 assigned to SR 2. Cost = 0
Brick 12 assigned to SR 2. Cost = 0
Brick 13 assigned to SR 2. Cost = 0
Brick 14 assigned to SR 2. Cost = 0
Brick 15 assigned to SR 1. Cost = 0
Brick 16 assigned to SR 3. Cost = 0
Brick 17 assigned to SR 3. Cost = 0
Brick 18 assigned to SR 3. Cost = 0
Brick 19 assigned to SR 4. Cost = 0
Brick 20 assigned to SR 4. Cost = 0
Brick 21 assigned to SR 4. Cost = 0
Brick 22 assigned to SR 4. Cost = 0


In [71]:
# Current solution
for sr in range(num_sr):
    d = sum([init_state_matrix[brick][sr] * distances_matrix[brick][sr] for brick in range(num_bricks)])
    w = sum([init_state_matrix[brick][sr] * index_value_matrix[brick][sr] for brick in range(num_bricks)])
    print(f"Distance for SR {sr + 1} : {d}")
    print(f"Workload for SR {sr + 1} : {w}") 

Distance for SR 1 : 19.3
Workload for SR 1 : 0.9507
Distance for SR 2 : 33.31
Workload for SR 2 : 1.3377
Distance for SR 3 : 9.99
Workload for SR 3 : 0.7048
Distance for SR 4 : 124.74
Workload for SR 4 : 1.0068


The current solution has several issues :
- The workload is not balanced between the SR (from 0.70 to 1.33)
- The total distance is not optimized (see results below)
- Also the distance that SR4 is traveling is the greatest by far (100km diff with the 2nd greatest distance), is this an issue ?

In [ ]:
solution = [[0 for x in range(num_sr)] for y in range(num_bricks)]
for brick in range(num_bricks):
    for sr in range(num_sr):
        if solver.boolean_value(x[brick, sr]):
            solution[brick][sr] = 1

In [70]:
# Solution found by the solver
for sr in range(num_sr):
    d = sum([solution[brick][sr] * distances_matrix[brick][sr] for brick in range(num_bricks)])
    w = sum([solution[brick][sr] * index_value_matrix[brick][sr] for brick in range(num_bricks)])
    print(f"Distance for SR {sr + 1} : {d}")
    print(f"Workload for SR {sr + 1} : {w}") 

Distance for SR 1 : 19.3
Workload for SR 1 : 0.9507
Distance for SR 2 : 28.8
Workload for SR 2 : 1.0848
Distance for SR 3 : 14.36
Workload for SR 3 : 0.9577
Distance for SR 4 : 124.74
Workload for SR 4 : 1.0068


This solution balances the workload by giving the 10th brick from SR2 to SR3.
The workload is now between 0.9507 and 1.0848.
It is also a minimal reassignation (one).
Still SR4 has 124km to travel.

### Workload between 0.96 and 1.1

In [87]:
model = cp_model.CpModel()

In [100]:
# Values for constraints
num_sr = 4
min_workload = 0.96
max_workload = 1.1
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
index_value_matrix_scaled = copy.deepcopy(index_value_matrix)
for i in range(len(index_value_matrix)):
    for j in range(len(index_value_matrix[0])):
        index_value_matrix_scaled[i][j] = int(index_value_matrix[i][j] * 10000)
distances_matrix = create_distances_matrix(distances)
distances_matrix_scaled = copy.deepcopy(distances_matrix)
for i in range(len(distances_matrix)):
    for j in range(len(distances_matrix[0])):
        distances_matrix_scaled[i][j] = int(distances_matrix[i][j] * 100)
hq_bricks = [[0,3],[1,13],[2,15],[3,21]]
num_bricks = len(distances_matrix)

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = model.new_bool_var(f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    model.add_exactly_one(x[brick, sr] for sr in range(num_sr))

# The cental brick for an SR cannot change
hq_vars = []
for hq in hq_bricks:
    hq_vars.append(x[hq[1],hq[0]])
model.add(sum(hq_vars) == 4)

# Balance workload (min and max)
for sr in range(num_sr):
    model.add(sum(index_value_matrix_scaled[brick][sr] * x[brick,sr] for brick in range(num_bricks)) >= int(min_workload * 10000))
    model.add(sum(index_value_matrix_scaled[brick][sr] * x[brick,sr] for brick in range(num_bricks)) <= int(max_workload * 10000))

In [101]:
objective_terms = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms.append(distances_matrix_scaled[brick][sr] * x[brick, sr])
model.minimize(sum(objective_terms))

In [102]:
solver = cp_model.CpSolver()
status = solver.solve(model)
print(status)

4


In [103]:
# Number of reassignments (upper bound for 2nd opti)
new_objective = 0

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Total cost = {solver.objective_value/100}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if solver.boolean_value(x[brick, sr]):
                new_objective += (1 - 2*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])
                print(
                    f"Brick {brick + 1} assigned to SR {sr + 1}."
                    + f" Cost = {distances_matrix[brick][sr]}"
                )
else:
    print("No solution found.")

Total cost = 182.83

Brick 1 assigned to SR 4. Cost = 21.12
Brick 2 assigned to SR 4. Cost = 17.33
Brick 3 assigned to SR 4. Cost = 12.25
Brick 4 assigned to SR 1. Cost = 0.0
Brick 5 assigned to SR 1. Cost = 3.07
Brick 6 assigned to SR 1. Cost = 1.22
Brick 7 assigned to SR 1. Cost = 2.8
Brick 8 assigned to SR 1. Cost = 2.87
Brick 9 assigned to SR 1. Cost = 3.8
Brick 10 assigned to SR 1. Cost = 12.35
Brick 11 assigned to SR 3. Cost = 2.97
Brick 12 assigned to SR 2. Cost = 22.01
Brick 13 assigned to SR 2. Cost = 3.28
Brick 14 assigned to SR 2. Cost = 0.0
Brick 15 assigned to SR 3. Cost = 1.11
Brick 16 assigned to SR 3. Cost = 0.0
Brick 17 assigned to SR 3. Cost = 1.08
Brick 18 assigned to SR 3. Cost = 1.54
Brick 19 assigned to SR 4. Cost = 25.4
Brick 20 assigned to SR 4. Cost = 23.21
Brick 21 assigned to SR 4. Cost = 25.43
Brick 22 assigned to SR 4. Cost = 0.0


In [104]:
# Adding hints as constraints
for brick in range(num_bricks):
    for sr in range(num_sr):
        model.add_hint(x[brick, sr], solver.Value(x[brick, sr]))

In [105]:
# Num of reassignments should be at least lower than the upper bound
for brick in range(num_bricks):
    model.add(sum(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr] for sr in range(num_sr)) <= round(new_objective))

In [106]:
# Minimizing reassignments
objective_terms_2 = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms_2.append(x[brick, sr] - 2*x[brick, sr]*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])
model.minimize(sum(objective_terms_2))

In [107]:
solver = cp_model.CpSolver()
status = solver.solve(model)
print(status)

4


In [108]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Total cost = {solver.objective_value/2}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if solver.boolean_value(x[brick, sr]):
                print(
                    f"Brick {brick + 1} assigned to SR {sr + 1}."
                    + f" Cost = {(1 - 2*init_state_matrix[brick][sr] + init_state_matrix[brick][sr])}"
                )
else:
    print("No solution found.")

Total cost = 3.0

Brick 1 assigned to SR 4. Cost = 0
Brick 2 assigned to SR 4. Cost = 0
Brick 3 assigned to SR 4. Cost = 0
Brick 4 assigned to SR 1. Cost = 0
Brick 5 assigned to SR 1. Cost = 0
Brick 6 assigned to SR 1. Cost = 0
Brick 7 assigned to SR 3. Cost = 1
Brick 8 assigned to SR 1. Cost = 0
Brick 9 assigned to SR 3. Cost = 0
Brick 10 assigned to SR 3. Cost = 1
Brick 11 assigned to SR 2. Cost = 0
Brick 12 assigned to SR 1. Cost = 1
Brick 13 assigned to SR 2. Cost = 0
Brick 14 assigned to SR 2. Cost = 0
Brick 15 assigned to SR 1. Cost = 0
Brick 16 assigned to SR 3. Cost = 0
Brick 17 assigned to SR 3. Cost = 0
Brick 18 assigned to SR 3. Cost = 0
Brick 19 assigned to SR 4. Cost = 0
Brick 20 assigned to SR 4. Cost = 0
Brick 21 assigned to SR 4. Cost = 0
Brick 22 assigned to SR 4. Cost = 0


In [110]:
solution = [[0 for x in range(num_sr)] for y in range(num_bricks)]
for brick in range(num_bricks):
    for sr in range(num_sr):
        if solver.boolean_value(x[brick, sr]):
            solution[brick][sr] = 1

In [111]:
# Solution found by the solver
for sr in range(num_sr):
    d = sum([solution[brick][sr] * distances_matrix[brick][sr] for brick in range(num_bricks)])
    w = sum([solution[brick][sr] * index_value_matrix[brick][sr] for brick in range(num_bricks)])
    print(f"Distance for SR {sr + 1} : {d}")
    print(f"Workload for SR {sr + 1} : {w}") 

Distance for SR 1 : 38.489999999999995
Workload for SR 1 : 0.9648
Distance for SR 2 : 6.789999999999999
Workload for SR 2 : 1.002
Distance for SR 3 : 25.28
Workload for SR 3 : 1.0264
Distance for SR 4 : 124.74
Workload for SR 4 : 1.0068


This time there are several reassignments:
- Brick 7 from SR1 to SR3
- Brick 10 from SR2 to SR3
- Brick 12 from SR2 to SR1
Once again the solution is balancing the workload of SR2 with the others.